## 動画におけるよりよい輪郭抽出手法
現在用いている階層を用いた輪郭抽出手法は文字が画面で切れていると階層が変化し認識できなくなってしまう。  
このソースコードでは切れていた時にfindcontourの参照階層を0から増やしていくことで問題の解決を図る。  
手法として、recognize_movie_cnnのコードを書き換え結果を比較する

In [0]:
#driveとの連携
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My Drive/School Project/System Design Project/recognize_num/

/content/drive/My Drive/School Project/System Design Project/recognize_num


In [0]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
from chainer import Sequential
import chainer.functions as F
import chainer.links as L
import chainer
from google.colab.patches import cv2_imshow
from pathlib import Path
from chainer import Chain, optimizers, Variable, serializers

まずは画像で

In [0]:
#動画ファイルの読み込み
cap = cv2.VideoCapture('/content/drive/My Drive/School Project/System Design Project/recognize_num/movie/num.mp4')
print(cap.isOpened())#読み込めたかどうか

True


In [0]:
#学習データの読み込み
class CNN(Chain):
    def __init__(self):
        super(CNN, self).__init__(
            conv1 = L.Convolution2D(1, 20, 5), # filter 5
            conv2 = L.Convolution2D(20, 50, 5), # filter 5
            l1 = L.Linear(800, 500),
            l2 = L.Linear(500, 500),
            l3 = L.Linear(500, 10, initialW=np.zeros((10, 500), dtype=np.float32))
        )
    def forward(self, x):
        h = F.max_pooling_2d(F.relu(self.conv1(x)), 2)
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 2)
        h = F.relu(self.l1(h))
        h = F.relu(self.l2(h))
        h = self.l3(h)
        return h
loaded_net = CNN()
chainer.serializers.load_npz('num_cnn.net', loaded_net)

In [0]:
#各設定
output_dir = Path("output_renew_fc")
output_dir.mkdir(exist_ok=True)
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # フレーム数を取得する。
n_digits = len(str(n_frames))  

In [0]:
while True: 
    ret, frame = cap.read()#1フレーム読み込み
    if ret == False:#フレームが読み込めなかったら出る
        break
    
    f_frame = frame.astype(np.uint8)#int変換
    bulr_img =cv2.GaussianBlur(f_frame,(15,15), 0)#平滑化
    gray_img = cv2.cvtColor(bulr_img, cv2.COLOR_RGB2GRAY)#グレースケール
    threshold_value=100 #閾値
    ret, thresh_img = cv2.threshold(gray_img, threshold_value, 255, cv2.THRESH_BINARY)#二値化
    #ここから輪郭抽出
    fc_img,contours, hierarchy = cv2.findContours(thresh_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    point_list = list()#矩形領域の座標リスト
    for layer in range(5):#layer = 階層の深さ
        for i in range(0, len(contours)):
            if len(contours[i]) > 0:
                if hierarchy[0][i][3] != layer:
                    continue
                rect = contours[i]
                x, y, w, h = cv2.boundingRect(rect)
                point_list.append([x,y,w,h])
        #矩形領域を抽出できたら
        if len(point_list) != 0:
            break
    imglist = list()
    for point in point_list:
        imglist.append(gray_img[point[1]:point[1]+point[3],point[0]:point[0]+point[2]])
    resize_imglist = list()
    for i in range(len(imglist)):
        #print(np.shape(imglist[i]))
        resize_imglist.append(cv2.resize(imglist[i], (28,28),0,0 ,cv2.INTER_NEAREST))
    rev_imglist=list()
    for i in range(len(resize_imglist)):
        rev_imglist.append(cv2.bitwise_not(resize_imglist[i]))
    flat_imglist=list()
    for i in range(len(rev_imglist)):
        flat_imglist.append(rev_imglist[i].flatten())

    arr = np.array(flat_imglist, dtype=np.float32)
    arr = arr.reshape((len(arr), 1, 28, 28))#整形してるよ
    if np.shape(arr)[0] != 0 :
        with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):
            prolist = loaded_net(arr)
        copy_img = f_frame.copy()
        con = 0
        for point in point_list :
            cv2.putText(copy_img, str(np.argmax(prolist[con,:].array)), (point[0], point[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 100, 0), 2, 8)
            cv2.rectangle(copy_img, (point[0], point[1]), (point[0] + point[2], point[1] + point[3]), (0, 100, 0), 2)
            con += 1     

        #copy_imgから画像を作る
        frame_no = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  #フレーム番号
        save_path = output_dir / f"frame_{frame_no:0{n_digits}d}.png"#パス
        cv2.imwrite(str(save_path), copy_img)#セーブ

    else:#認識に失敗したとき
        frame_no = int(cap.get(cv2.CAP_PROP_POS_FRAMES))  #フレーム番号
        save_path = output_dir / f"frame_{frame_no:0{n_digits}d}.png"#パス
        cv2.imwrite(str(save_path), f_frame)#セーブ


cap.release()  

In [0]:
#画像ファイルを読み込み動画にして保存する
img_paths = Path("output_renew_fc") / "frame_%03d.png"

# VideoCapture を作成する。
rec_video = cv2.VideoCapture(str(img_paths))
width = int(rec_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(rec_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = 15
print(f"width: {width}, height: {height}, fps: {fps}")

# VideoWriter を作成する。
fourcc = cv2.VideoWriter_fourcc(*"DIVX")
writer = cv2.VideoWriter("output_renew_fc.avi", fourcc, fps, (width, height))

while True:
    # 1フレームずつ取得する。
    ret, frame = rec_video.read()
    if not ret:
        break  # フレームの取得に失敗または動画の末尾に到達した場合

    writer.write(frame)  # フレームを書き込む。

writer.release()
rec_video.release()

width: 720, height: 480, fps: 15


#結果
完璧！！！
きれいな結果が出ている